In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/nfl.csv")
df.head()

,3d%,3d%_allowed,TOP,date,first_downs,first_downs_allowed,ha,margin,opp,pass_yards,...,rush_yards,rush_yards_allowed,sacked,sacks,takeaways,team,total_points,total_yards,total_yards_allowed,turnovers
0,0.307692,0.285714,28.350000,9/10/2009,18,19,away,-3,PIT,234,...,86,36,1,4,3,TEN,3,320,357,2
1,0.285714,0.307692,36.183333,9/10/2009,19,18,home,3,TEN,321,...,36,86,4,1,2,PIT,3,357,320,3
2,0.363636,0.400000,29.116667,9/13/2009,16,19,away,-12,ATL,163,...,96,68,4,2,0,MIA,26,259,281,4
3,0.400000,0.363636,30.883333,9/13/2009,19,16,home,12,MIA,213,...,68,96,2,4,4,ATL,26,281,259,0
4,0.250000,0.333333,26.550000,9/13/2009,10,16,away,5,CIN,227,...,75,86,3,3,2,DEN,19,302,307,0


In [3]:
# Dummify ha where away is 0 and home is 1
df["ha"] = df["ha"].map(lambda x: 0 if x == "away" else 1)

In [4]:
# Grab dummy variables
dummy_vars = pd.get_dummies(df[["opp", "team"]])
dummy_vars.head()

,opp_ARI,opp_ATL,opp_BAL,opp_BUF,opp_CAR,opp_CHI,opp_CIN,opp_CLE,opp_DAL,opp_DEN,...,team_NYG,team_NYJ,team_OAK,team_PHI,team_PIT,team_SEA,team_SFO,team_TBB,team_TEN,team_WAS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Add our dummy columns into the features df
df[list(dummy_vars.columns)] = dummy_vars

In [6]:
# define the features and label
X = df.drop(["result", "date", "opp", "team", "margin", "points", "points_allowed", "total_points"], axis=1)
y = df['result']
print(X.shape, y.shape)

(4806, 82) (4806,)


In [7]:
X.head()

,3d%,3d%_allowed,TOP,first_downs,first_downs_allowed,ha,pass_yards,pass_yards_allowed,penalty_yards,plays,...,team_NYG,team_NYJ,team_OAK,team_PHI,team_PIT,team_SEA,team_SFO,team_TBB,team_TEN,team_WAS
0,0.307692,0.285714,28.350000,18,19,0,234,321,28,61,...,0,0,0,0,0,0,0,0,1,0
1,0.285714,0.307692,36.183333,19,18,1,321,234,60,70,...,0,0,0,0,1,0,0,0,0,0
2,0.363636,0.400000,29.116667,16,19,0,163,213,27,56,...,0,0,0,0,0,0,0,0,0,0
3,0.400000,0.363636,30.883333,19,16,1,213,163,35,65,...,0,0,0,0,0,0,0,0,0,0
4,0.250000,0.333333,26.550000,10,16,0,227,221,39,51,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X.columns

Index(['3d%', '3d%_allowed', 'TOP', 'first_downs', 'first_downs_allowed', 'ha',
       'pass_yards', 'pass_yards_allowed', 'penalty_yards', 'plays',
       'rush_yards', 'rush_yards_allowed', 'sacked', 'sacks', 'takeaways',
       'total_yards', 'total_yards_allowed', 'turnovers', 'opp_ARI', 'opp_ATL',
       'opp_BAL', 'opp_BUF', 'opp_CAR', 'opp_CHI', 'opp_CIN', 'opp_CLE',
       'opp_DAL', 'opp_DEN', 'opp_DET', 'opp_GBP', 'opp_HOU', 'opp_IND',
       'opp_JAX', 'opp_KCC', 'opp_LAC', 'opp_LAR', 'opp_MIA', 'opp_MIN',
       'opp_NEP', 'opp_NOS', 'opp_NYG', 'opp_NYJ', 'opp_OAK', 'opp_PHI',
       'opp_PIT', 'opp_SEA', 'opp_SFO', 'opp_TBB', 'opp_TEN', 'opp_WAS',
       'team_ARI', 'team_ATL', 'team_BAL', 'team_BUF', 'team_CAR', 'team_CHI',
       'team_CIN', 'team_CLE', 'team_DAL', 'team_DEN', 'team_DET', 'team_GBP',
       'team_HOU', 'team_IND', 'team_JAX', 'team_KCC', 'team_LAC', 'team_LAR',
       'team_MIA', 'team_MIN', 'team_NEP', 'team_NOS', 'team_NYG', 'team_NYJ',
       'team_OA

In [9]:
# only select one row
X = X.iloc[[0]]

In [21]:
X

,3d%,3d%_allowed,TOP,first_downs,first_downs_allowed,ha,pass_yards,pass_yards_allowed,penalty_yards,plays,...,team_NYG,team_NYJ,team_OAK,team_PHI,team_PIT,team_SEA,team_SFO,team_TBB,team_TEN,team_WAS
0,0.307692,0.285714,28.35,18,19,0,234,321,28,61,...,0,0,0,0,0,0,0,0,1,0


In [11]:
oriented_df = X.to_dict(orient="records")

In [12]:
default_input_dict = oriented_df[0]

In [22]:
columns = list(X.columns)

In [14]:
zeros = np.zeros(shape=(1,82))

In [15]:
new_df = pd.DataFrame(zeros, columns=columns)

In [16]:
new_df

,3d%,3d%_allowed,TOP,first_downs,first_downs_allowed,ha,pass_yards,pass_yards_allowed,penalty_yards,plays,...,team_NYG,team_NYJ,team_OAK,team_PHI,team_PIT,team_SEA,team_SFO,team_TBB,team_TEN,team_WAS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data = {}

In [17]:
oriented = new_df.to_dict(orient="records")

In [33]:
model_input_dict = {'team': 'ARI', 'opponent': 'DAL', 'third': '2345', 'third-allowed': '68', 'top': '6578', 'first-downs': '2345', 'first-downs-allowed': '0790', 'ha': '2345', 'pass-yards': '789', 'pass-yards-allowed': '789', 'penalty-yards': '2345', 'plays': '6', 'rush-yards': '4573', 'rush-yards-allowed': '2456', 'sacked': '3456', 'sacks': '34578', 'takeaways': '468', 'total-yards': '234', 'total-yards-allowed': '572', 'turnovers': '245'}

In [34]:
for key, value in model_input_dict.items():
    
    print(key, value)

team ARI
opponent DAL
third 2345
third-allowed 68
top 6578
first-downs 2345
first-downs-allowed 0790
ha 2345
pass-yards 789
pass-yards-allowed 789
penalty-yards 2345
plays 6
rush-yards 4573
rush-yards-allowed 2456
sacked 3456
sacks 34578
takeaways 468
total-yards 234
total-yards-allowed 572
turnovers 245


In [35]:
features = ["Team", "Opponent", "Third", "Third Allowed", "TOP", "First Downs",
               "First Downs Allowed", "Home or Away", "Pass Yards", "Pass Yards Allowed",
               "Penalty Yards", "Plays", "Rush Yards", "Rush Yards Allowed", "Sacked", "Sacks",
               "Takeaways", "Total Yards", "Total Yards Allowed", "Turnovers"]

form_names = ["team", "opponent", "third", "third-allowed", "top", "first-downs",
                  "first-downs-allowed", "ha", "pass-yards", "pass-yards-allowed", "penalty-yards",
                  "plays", "rush-yards", "rush-yards-allowed", "sacked", "sacks", "takeaways",
                  "total-yards", "total-yards-allowed", "turnovers"]

placeholders = ["team", "opponent", "third", "third-allowed", "top", "first-downs",
                    "first-downs-allowed", "ha", "pass-yards", "pass-yards-allowed", "penalty-yards",
                    "plays", "rush-yards", "rush-yards-allowed", "sacked", "sacks", "takeaways",
                    "total-yards", "total-yards-allowed", "turnovers"]

In [37]:
form_df = pd.DataFrame({
    "feature": features,
    "form_name": form_names,
    "placeholder": placeholders
})

In [39]:
form_df.head()

,feature,form_name,placeholder
0,Team,team,team
1,Opponent,opponent,opponent
2,Third,third,third
3,Third Allowed,third-allowed,third-allowed
4,TOP,top,top


In [40]:
form_data = form_df.to_dict(orient="records")

In [41]:
form_data

[{'feature': 'Team', 'form_name': 'team', 'placeholder': 'team'},
 {'feature': 'Opponent', 'form_name': 'opponent', 'placeholder': 'opponent'},
 {'feature': 'Third', 'form_name': 'third', 'placeholder': 'third'},
 {'feature': 'Third Allowed',
  'form_name': 'third-allowed',
  'placeholder': 'third-allowed'},
 {'feature': 'TOP', 'form_name': 'top', 'placeholder': 'top'},
 {'feature': 'First Downs',
  'form_name': 'first-downs',
  'placeholder': 'first-downs'},
 {'feature': 'First Downs Allowed',
  'form_name': 'first-downs-allowed',
  'placeholder': 'first-downs-allowed'},
 {'feature': 'Home or Away', 'form_name': 'ha', 'placeholder': 'ha'},
 {'feature': 'Pass Yards',
  'form_name': 'pass-yards',
  'placeholder': 'pass-yards'},
 {'feature': 'Pass Yards Allowed',
  'form_name': 'pass-yards-allowed',
  'placeholder': 'pass-yards-allowed'},
 {'feature': 'Penalty Yards',
  'form_name': 'penalty-yards',
  'placeholder': 'penalty-yards'},
 {'feature': 'Plays', 'form_name': 'plays', 'placehold